In [2]:
"""
======================================
Example 8: Fluctuation Field Generation
======================================

This example demonstrates the utilities for generating synthetic turbulence, which can be either
from a pre-trained DRD model, or based on some well-known spectra models. ``DRDMannTurb``
provides several utilities for plotting the resulting fields through Plotly, which can be done
in several contexts as well as utilities for saving to VTK for downstream analysis in, e.g.,
ParaView.
"""  # noqa

#######################################################################################
#   .. centered::
#       This example may take a few seconds to load. Please be patient if using
#       Plotly, as it requires some time to render 3D graphics.
#

#######################################################################################
# Import packages
# ---------------
#
# First, we import the packages we need for this example.
from pathlib import Path

import numpy as np
import torch

from drdmannturb.fluctuation_generation import (
    FluctuationFieldGenerator,
)

path = Path().resolve()

device = "cuda" if torch.cuda.is_available() else "cpu"

if torch.cuda.is_available():
    torch.set_default_tensor_type("torch.cuda.FloatTensor")

#######################################################################################
# Setting Physical Parameters
# ---------------------------
# Here, we set the physical parameters of the environment in which the synthetic wind field is generated:
# the friction velocity :math:`u_\mathrm{red} = 11.4` roughness height :math:`z_0=0.02` and reference height
# of :math:`90`. The physical domain is determined by dimensions in 3D as well as the discretization
# size (grid levels) in each dimension.
z0 = 0.02
zref = 90
uref = 11.4
ustar = uref * 0.41 / np.log(zref / z0)
plexp = 0.2  # power law exponent
windprofiletype = "PL"  # choosing power law, use log with "LOG" here instead

L = 0.593 * zref  # length scale
Gamma = 3.89  # time scale
sigma = 0.052  # magnitude (σ = αϵ^{2/3})

Lx = 720
Ly = 64
Lz = 64

nBlocks = 2
grid_dimensions = np.array([Lx / 4, Ly, Lz])

grid_levels = np.array([6, 4, 4])

seed = None

#######################################################################################
# Generating Fluctuation Field from Mann Model
# --------------------------------------------
# Fluctuation fields are generated block-by-block, rather than over the domain entirely.
# Please see section V, B of the original DRD paper for further discussion. Here, we will use 4 blocks.

Type_Model = "Mann"  ### 'Mann', 'VK', 'DRD'

#######################################################################################
# Physical Parameters
# -------------------
# The Mann model requires three parameters, length scale, time scale, and spectrum amplitude scale,
# which are defined above
#
gen_mann = FluctuationFieldGenerator(
    ustar,
    zref,
    grid_dimensions,
    grid_levels,
    length_scale=L,
    time_scale=Gamma,
    energy_spectrum_scale=sigma,
    model=Type_Model,
    seed=seed,
)

fluctuation_field_mann = gen_mann.generate(nBlocks, zref, uref, z0, windprofiletype, plexp)

In [3]:
fluctuation_field_mann

array([[[[-1.26418526e+00, -5.48676155e-01,  1.56919548e+00],
         [ 4.52656648e+00, -8.18122431e-01,  1.15132566e+00],
         [ 5.35569342e+00, -5.86001305e-01,  9.11681559e-01],
         ...,
         [ 1.16605115e+01,  7.55232975e-01,  1.03009062e+00],
         [ 1.14395726e+01, -8.62886954e-02,  1.63781037e+00],
         [ 1.11541301e+01,  7.28256706e-01,  1.88959577e+00]],

        [[-1.19770278e+00, -1.38913649e-01,  1.14011247e+00],
         [ 4.34933215e+00, -1.14079818e-01,  5.15747233e-01],
         [ 4.65863744e+00, -1.43166874e-01,  6.05436018e-01],
         ...,
         [ 1.12456055e+01,  7.03487243e-01,  1.01797049e+00],
         [ 1.11944660e+01,  1.98842664e-02,  1.01126293e+00],
         [ 1.10004718e+01,  5.58028056e-01,  5.42844448e-01]],

        [[-4.84811430e-01,  1.34054670e-01,  1.08350092e+00],
         [ 4.53396183e+00, -3.18852244e-03,  7.41778280e-01],
         [ 4.35715927e+00, -6.70453553e-01,  9.52652060e-01],
         ...,
         [ 1.05159371e+0

In [4]:
# Print min and max of u, v, w = [:, : ,:, 0-2]
print("u (min, max):", fluctuation_field_mann[..., 0].min(), fluctuation_field_mann[..., 0].max())
print("v (min, max):", fluctuation_field_mann[..., 1].min(), fluctuation_field_mann[..., 1].max())
print("w (min, max):", fluctuation_field_mann[..., 2].min(), fluctuation_field_mann[..., 2].max())


u (min, max): -3.2462094219393007 14.075215228391055
v (min, max): -2.4219225765251755 3.310158559612362
w (min, max): -2.9991249723122184 3.280022196402824


In [7]:
gen_mann.save_to_vtk("STORM_80_fluctuation_field.vtk")